<h1>機械学習モデル作成用</h1>

#---------------------------------------------------------------------------------------------------#

既にcfar,スペクトログラム画像の作成を終えている段階で使用可能。

<h2>初期設定</h2>

<h3>1.各種ライブラリのインポート</h3>

In [ ]:
import sys
import os
import gc
import re
import math
import tqdm
import itertools
import shutil
import pickle
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from scipy import signal
%matplotlib inline

<h3>2.whale_call_packageディレクトリを格納している絶対パスを記述.</h3>
windows,macでそれぞれ用意しているが不要な方はコメントアウト

In [ ]:
#windows
sys.path.append("C:/Users")

#mac
sys.path.append("/Users/nakahigashikazuo3/Desktop")

<h3>3.whale_call_packageをインポート</h3>

In [ ]:
from whale_call_package import whale_call as wh
from whale_call_package import parameter_setting as para

In [ ]:
#OBSナンバーを選択。数字6桁以上は避けてほしい。
obs = ''

<h3>4.データを出力するディレクトリ、データが格納されているディレクトリを記述。</h3>不要な方をコメントアウト

datadirectoryの方をobsデータの入っている絶対パスに変更するだけでOK。

In [ ]:
#windows
workdir_base = f'D:/whale/data/WhaleCallField'
preworkdirectory=f'{workdir_base}/{obs}'
datadirectory=f'D:/whale/data/origindata/{obs}'
#datadirectory=f'D:/whale/JB17a/{obs}'

#mac
preworkdirectory=f'/Volumes/LaCie/WhaleCallField/{obs}'
datadirectory=f'/Volumes/LaCie/JS18a/data/{obs}'

<h3>5.パラメータ変更用に作業ディレクトリ下にディレクトリを設定する</h3>

In [ ]:
currentdirectory = '5' #任意の名前をつける.数字４文字以上は不具合が出る
parameter_number = '2' #whale_call_package内のparameter_setting.pyを確認

In [ ]:
#環境設定をインスタンス化し、作業ディレクトリを作成、waveファイルのリストを取得する
workdirectory=preworkdirectory+'/'+currentdirectory
whale = wh.SetField(workdirectory=workdirectory, datadirectory=datadirectory)
whale.make_directory()
filelist = whale.get_wavfilelist()

<h3>6.処理を行うwavファイルを指定する場合以下を実行する。</h3>不要ならば全てコメントアウト

<h4>1．任意で指定する。</h4>

In [ ]:
#filelist = whale.pick_wavfilelist(filelist=filelist, month=["1904"])
#"1808", "1809", "1810", "1811", "1812", "1901", , "1903", "1904","1905","1906","1907"

<h4>2．ランダムで指定する。</h4>

In [ ]:
#filelist = whale.random_pick_wavfilelist(filelist=filelist, pick_num=5)

<h4>3.不要なファイルを削除する。</h4>

In [ ]:
#filelist = whale.del_wavfile(filelist=filelist, del_list=['180725','180726'])

<h4>4.最初の２日を排除する。</h4>

In [ ]:
filelist = filelist[2:]

取得したファイル一覧

In [ ]:
filelist

In [ ]:
#各パラメータセッティング
sr, n_fft, hop_length, fmin, fmax, fpass, fstop, gpass, gstop, num_train, num_guard, rate_fa = para.para(parameter_number)

In [ ]:
#各パラメータのログ出力
para.make_parameter_log(workdirectory=workdirectory, sr=sr, n_fft=n_fft, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

In [ ]:
cfar = wh.Cfar(workdirectory=workdirectory, datadirectory=datadirectory, sr=sr, n_fft=n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)
spec = wh.MakeSpectrogram(workdirectory=workdirectory, datadirectory=datadirectory, sr=sr, n_fft=n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

#---------------------------------------------------------------------------------------------------#

<h2>CNNでノイズ、センタリングを除去するためのモデルを作成する</h2>

1.GPUを認識しているかを確認する。認識している場合、GPUの名前等が表示される

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2.各パラメータと分類リストを設定する

In [ ]:
# 学習率
learning_rate = 1e-3
# エポック
epochs = 20
# バッチサイズ
batch_size = 256
#画像サイズ
image_size =128
#分類リスト
dir_list = ['centering', 'noise', 'call', 'n_call']
with open(f'{workdirectory}/cfar/all/call_list.txt', 'w', encoding='utf-8') as f:
    pass

In [ ]:
cnn = wh.Cnn(workdirectory=workdirectory, datadirectory=datadirectory, sr=sr, n_fft=n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

3.各日から訓練用データに使用する鳴音画像をランダムに抽出し、抽出した画像をテキストファイルにメモする

In [ ]:
cnn.prepare_cnn(sr=sr, dir_list=dir_list)
cnn.set_call_list()

4.学習させたいデータをrandomディレクトリからtrain_iventディレクトリ内の各分類に目視で判断し、移動させる

In [ ]:
train_dir, val_dir, backup_dir = cnn.dir_set(dir_list=dir_list)

In [ ]:
train_dir, val_dir, backup_dir

5.CNNモデルを作成し、精度を表示する.

In [ ]:
if sentering_flag:
    cnn.cnn_el_sentering(learning_rate=learning_rate, epochs=epochs, batch_size=batch_size, image_size=image_size, train_dir=train_dir, val_dir=val_dir, backup_dir=backup_dir)

6.作成したモデルを元にテストする

In [ ]:
test_img_list = f'{workdirectory}/cfar/all/random'
sep_img_list =  f'{workdirectory}/cfar/all/fig_convert'
model_name = 'my_model.h5'
days= os.listdir(f'{workdirectory}/cfar')
days.remove('all')
days.remove('pick_list.txt')

In [ ]:
all_result = [0]*4
for i in days:
    all_result = cnn.sep_cnn_noise(test_img_list=test_img_list,detected_dir=f'{workdirectory}/cfar/all/detected', model_name=model_name, days=i, all_result=all_result)
cnn.return_target(train_dir=train_dir, val_dir=val_dir) #テスト用に分離した画像を元の訓練用データに戻す

In [ ]:
all_result

7.detectedディレクトリ内の画像の誤分類を手動で修正し、以下を実行することで訓練用データに加える。

In [ ]:
cnn.merge_detect(train_dir=train_dir)

8.手順の3-7を混同行列を参考に精度を検証しながら繰り返す。